In [3]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
#from BCEmbedding.tools.langchain import BCERerank
from FlagEmbedding import FlagReranker
import pandas as pd
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain.retrievers import ContextualCompressionRetriever
import csv
import os
import time
import pickle
from zhipuai import ZhipuAI
import csv
import json

In [4]:
model_name = "BAAI/bge-large-zh-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
embedding = HuggingFaceBgeEmbeddings(
                model_name=model_name,
                model_kwargs=model_kwargs,
                encode_kwargs=encode_kwargs,
                query_instruction="为这个句子生成表示以用于检索相关文章:"
            )

In [5]:
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up 
db = FAISS.load_local('tu_document_1024_512',embedding,allow_dangerous_deserialization = True)
retriever=db.as_retriever( search_kwargs={"k":9})

In [6]:
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(
        model_name="GLM-4-PLus",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4",
        openai_api_key="3138eb04ab176ee38855bc7bd5883868.RZfclCazuZkKXA1f",
        streaming=False,)

/home/vipuser/miniconda3/envs/vllm/lib/python3.11/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [7]:
def format_docs(docs):
    return "".join(doc.page_content for doc in docs)
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt 
    | llm
    | StrOutputParser()
)
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)
extracted_data = []

In [8]:
with open('data/val.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        question = data['input_field']
        prompt="以下问题只关于TuGraph-DB\n"+ question +"\n要求：直接回答结果，如果不知道回答不知道，不要添加额外内容，不要进行推测。回答长度不要超过100字。"#提示词这块是需要工程化来做的
        result = rag_chain_with_source.invoke(question)
        extracted_info = {"id": data["id"],"output_field": result['answer']}
        extracted_data.append(extracted_info)
        print(question,result['answer'])
        print('\n')
#print(extracted_data)
# 将提取的数据写入到answer.jsonl文件中
with open('result/val_ans.jsonl', 'w',encoding='utf-8') as f:
    for item in extracted_data:
        # 将每个字典转换为JSON格式的字符串，并写入文件
        json_line = json.dumps(item,ensure_ascii=False) + '\n'
        f.write(json_line)
print("数据已写入到val_ans.jsonl文件中。")

RPC 及 HA 服务中，verbose 参数的设置有几个级别？ RPC 及 HA 服务中，verbose 参数的设置有三个级别，分别为 0、1、2。值越大，日志输出信息越详细。默认值为 1。


在磁盘IO监控的配置中，当哪个值大于10000时会触发危急颜色模式？ 在磁盘IO监控的配置中，当值大于10000时会触发危急颜色模式。这个阈值设置在"thresholds"部分，具体为`"colorMode": "critical", "op": "gt", "value": 10000`。这表示当磁盘IO超过10000时，监控界面会显示危急颜色。


`FieldData` 类中的函数 `IsReal()` 是用来查询什么类型的数据？ `FieldData` 类中的函数 `IsReal()` 是用来查询数据是否为实数类型，即是否为 `float` 或 `double` 类型。这可以从 `type` 属性的可能值中推断出来，其中包括 `float` 和 `double` 作为实数类型。其他类型如 `int8`, `int16`, `string` 等则不是实数类型。


如果成功修改一个用户的描述，应返回什么状态码？   如果成功修改一个用户的描述，应返回状态码 200。这个状态码表示请求已成功处理。在API文档中明确指出，修改用户描述的URI为`/user/{user_name}/description`，使用PUT方法，成功响应的代码是200。


边关联的两个点的字段，一定是点的主键吗？ 边关联的两个点的字段不一定是点的主键。第二个和第三个参数中配置的起点和终点的主键字段只是起占位和区别作用，并非实际的主键字段名。实际的主键字段名需要在创建点Label时通过`primary`字段指定。


OutEdgeIterator 类的 Delete 方法执行什么操作？ OutEdgeIterator 类的 Delete 方法用于删除当前指向的出边。删除后，迭代器会指向下一个按 (label, tid, dst, eid) 排序的出边，如果没有更多出边，则迭代器变为无效。在无效事务、无效迭代器或只读事务中调用此方法会抛出异常。


TuGraph-DB的日志等级如何调整？ TuGraph-DB的日志等级通过配置文件中的`verbose`选项进行调整。`verbose`选项分为

In [9]:
with open('data/test1.jsonl', 'r') as file:
    for line in file:
        data = json.loads(line)
        question = data['input_field']
        prompt="以下问题只关于TuGraph-DB\n"+ question +"\n要求：直接回答结果，如果不知道回答不知道，不要添加额外内容，不要进行推测。回答长度不要超过100字。"#提示词这块是需要工程化来做的
        result = rag_chain_with_source.invoke(question)
        extracted_info = {"id": data["id"],"output_field": result['answer']}
        extracted_data.append(extracted_info)
        print(question,result['answer'])
        print('\n')
#print(extracted_data)
# 将提取的数据写入到answer.jsonl文件中
with open('result/answer2.jsonl', 'w',encoding='utf-8') as f:
    for item in extracted_data:
        # 将每个字典转换为JSON格式的字符串，并写入文件
        json_line = json.dumps(item,ensure_ascii=False) + '\n'
        f.write(json_line)
print("数据已写入到answer.jsonl文件中。")

在添加边时，如果指定的值不包含在value_dict中将如何处理？ 在添加边时，如果指定的值不包含在`value_dict`中，这些字段将被视为`null`。这意味着未在`value_dict`中指定的字段不会影响边的创建，但它们在边的属性中将不存在或为空。文档中明确指出，不在`value_dict`中的字段被认为是`null`。


在使用LIMIT子句时，如果查询数据库中前两个人的名字，返回的名字是什么？ 在使用`LIMIT`子句查询数据库中前两个人的名字时，返回的名字是"Christopher Nolan"和"Corin Redgrave"。这是根据`MATCH (n:Person) RETURN n.name LIMIT 2;`查询结果得出的。


如何查询数据库中现有角色及其相关信息？ 要查询数据库中现有角色及其相关信息，可以使用以下方法：

1. 通过API请求：发送GET请求到`/role/`端点，管理员权限下可获取所有角色及其信息。
2. 使用命令：在数据库中执行`CALL dbms.security.listRoles()`命令，列出所有角色及其详细信息。

示例API请求：`GET http://localhost:7070/role`，需包含适当的认证信息。


tugraph可以最多创建多少点边和点边上最多创建多少属性？ TuGraph每个图项目最多可以创建2^(40)个点，两个点之间最多可以创建2^(32)条边。每个点或边类型的属性数量上限为1024个。


我下载了4.3.2镜像，启动成功了，进入容器后没有 setup.sh 脚本是改变目录了吗？ 根据提供的上下文，没有直接提及setup.sh脚本的信息。启动命令和参数配置主要集中在备份、编译等操作上，并未涉及setup.sh脚本的具体位置或是否存在。因此，无法确定是否因为改变目录导致找不到setup.sh脚本。建议检查镜像内的目录结构或查阅相关文档确认setup.sh脚本的位置。


lgraph_server -d start的方式启动，不是会在pwd路径下生成pid文件吗？这个pid文件有参数能控制路径吗？ 是的，使用 `lgraph_server -d start` 命令启动 TuGraph 服务器时，会在当前工作目录（pwd）下生成一个 `.pid` 文件，用于记录进程 ID。目